In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import time
import polars as pl
import tqdm


In [ ]:
from dspy.api import get_api
from dspy.api.bybit import Config

config = Config()
s = get_api('bybit', config=config)


# <font color="grey">ByBit API</font>

In [ ]:
config

In [4]:
s = ByBitManager(config)

In [5]:
symbols = ['BTCUSDT', 'ETHUSDT']

### Prices and volumes

In [ ]:
mid = s.get_mid(symbols[1])
print("Last price: ", mid)

In [ ]:
ask = s.get_ask(symbols[0])
bid = s.get_bid(symbols[0])
print("Bid: ", bid)
print("Ask: ", ask)

### Account info

In [ ]:
balance = s.get_wallet_balance()
print("Balance: ", balance)

In [ ]:
fees = s.get_fees('BTCUSDT')
print("Fees: ", fees)

### Position info

In [ ]:
s.s.get_positions(category='linear', 
            symbol=symbols[0],
            )['result']['list'][0]

In [ ]:
pos = s.get_position(symbols[0])
pos is None

### Trading

In [ ]:
# Place market order
res = s.place_order('BTCUSDT', 0.005)
res

In [ ]:
balance = s.get_wallet_balance()
print("Balance: ", balance)

In [ ]:
pos = s.get_position(symbols[0])
pos

In [ ]:
entry = pos['entry_price']
entry

In [ ]:
entry*float(fees[0])*0.005

In [ ]:
liq_price = s.get_bid('BTCUSDT')[0]
liq_price-entry

In [ ]:
pos = s.get_positions(symbols)['BTCUSDT']
diff = (pos['mark_price']-pos['entry_price'])*0.005
diff2 = pos['unrealized_pnl']

In [ ]:
(diff-diff2)<1e-8

In [ ]:
(pos['realized_pnl']+fees[0]*pos['entry_price']*0.005)<1e-8

In [ ]:
# Close position
response = s.close_positions(['BTCUSDT'])
response

In [ ]:
# Limit orders
price = s.get_mid(config.symbol)
response = s.place_order(config.symbol,
                         -0.002,
                         price+10,
                         type='Limit')
print(response)

In [ ]:
price = s.get_mid(symbols[0])
response = s.place_order(symbols[0],
                         -0.001,
                         price+100,
                         type='Limit')
print(response)

In [ ]:
response = s.replace_order(symbols[0], response['order_id'], -0.001, price+100)
print(response)

In [ ]:
resp = s.cancel_order(symbols[0], response['order_id'])
print(resp)

In [ ]:
# Batch orders
response = s.place_batch_order(symbols[0], [-0.001, -0.001], [price+50, price+100])

In [ ]:
s.cancel_batch_order(symbols[0], [x['order_id'] for x in response])

In [ ]:
s.cancel_all_orders([symbols[0]])

In [ ]:
s.close_positions(['BTCUSDT'])

In [ ]:
mm(s, config)

In [ ]:
s.cancel_all_orders(config.symbol)
last = s.get_mid(config.symbol)
price_range = round(config.range * last, 2)
balance = s.get_wallet_balance()
available_equity = (balance * config.equity) / last
# Arithmetic grid
nn = 2*available_equity / (config.num_orders * (config.num_orders + 1))
long_qtys = np.round([nn * i for i in reversed(range(1, config.num_orders + 1))], 
                             decimals=3)
short_qtys = np.round([-i for i in long_qtys][::-1], decimals=3)
long_prices = np.round(
            np.linspace(last - price_range/2, 
                        last - nn, 
                        config.num_orders), 
            decimals=1
            )
short_prices = np.round(
            np.linspace(last + nn, 
                        last + price_range/2, 
                        config.num_orders), 
            decimals=1
            )


In [ ]:
long_order_ids = s.place_batch_order(config.symbol, long_qtys, long_prices)
short_order_ids = s.place_batch_order(config.symbol, short_qtys, short_prices)

In [ ]:
s.cancel_all_orders(config.symbol)

In [ ]:
mm(s, config)

In [ ]:
p = s.get_position(config.symbol)
p

In [ ]:
int(p['mark_price']*(1+0.005))

In [ ]:
np.linspace(0,1,10)